In [1]:
%tensorflow_version 2.x  

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from IPython.display import clear_output 

import tensorflow as tf 
from tensorflow import feature_column 

In [2]:
# Load our training and test data 
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training dataset
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing dataset
#print(dftrain.head())

y_train = dftrain.pop('survived') # pop survived data from dataframe, this is the data we want to isolate and do the regression on 
y_eval = dfeval.pop('survived') 

In [3]:
# Observe and get to know structure and characteristics of data (Not Required to train Model)

# dftrain.describe() # prints out stats about data such as mean, std, etc.
dftrain.info() #prints out all the columns in data and the data type used to represent each (helpful for determining categorical/numerical)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sex                 627 non-null    object 
 1   age                 627 non-null    float64
 2   n_siblings_spouses  627 non-null    int64  
 3   parch               627 non-null    int64  
 4   fare                627 non-null    float64
 5   class               627 non-null    object 
 6   deck                627 non-null    object 
 7   embark_town         627 non-null    object 
 8   alone               627 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 44.2+ KB


In [ ]:
dftrain.hist(bins=20)
# print(type(dftrain))
# for item in NUMERICAL_COLUMNS: 
#   dftrain[item].hist(bins=20)

In [ ]:
# Observe and get to know structure and characteristics of data (Not Required to train Model)
dftrain['sex'].value_counts().plot(kind='bar')


In [ ]:
# Observe and get to know structure and characteristics of data (Not Required to train Model)
dftrain['class'].value_counts().plot(kind='bar')

In [6]:
# setup features for model 

# CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
# NUMERICAL_COLUMNS = ['age', 'fare']

CATEGORICAL_COLUMNS = ['sex', 'class', 'deck', 'embark_town', 'alone']
NUMERICAL_COLUMNS = ['age', 'fare', 'n_siblings_spouses', 'parch']

feature_columns = [] 

# iterate through categorical columns
for feature_name in CATEGORICAL_COLUMNS: 
    vocabulary = dftrain[feature_name].unique() # get all the unique values for each category (ex. Sex would M/F)
    new_feature = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary) # creates a feature for a category and all the unique entries
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)) # add the feature to the list  

# iterate through numerical columns 
for feature_name in NUMERICAL_COLUMNS: 
    new_feature = tf.feature_column.numeric_column(feature_name, dtype = tf.float32) # create a feature with the category and a float value
    feature_columns.append(new_feature) # give feature name and data type for numeric # add the features to the list

In [7]:
# Create an input function that will convert our pandas dataframe into a tf.data.dataset obj, source: https://www.tensorflow.org/tutorials/estimator/linear
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=100):
  def input_function():  # Nested function 
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    # example of repeat and batch method are used:
    # repeat: [1, 2, 3].repeat(3) -> [1, 2, 3, 1, 2, 3, 1, 2, 3]
    # batch: dataset = tf.data.Dataset.range(8).batch(3) -> [[0, 1, 2], [3, 4, 5], [6, 7]]
    return ds  # return batch 
  return input_function  # return dataset object 

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [ ]:
# Create model with feature columns 
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
# train model with function 
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model accuracy on tetsing data

clear_output()
print(result['accuracy'])  

In [ ]:
predictions = list(linear_est.predict(eval_input_fn)) # should be an object containing probabilities, class_ids...

probs = pd.Series([pred['probabilities'][1] for pred in predictions])
probs.plot(kind='hist', bins=20, title='predicted probabilities')
